![Iris](https://miro.medium.com/max/700/1*f6KbPXwksAliMIsibFyGJw.png)

In [7]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn import functional as F
from torch import nn
from torch.optim import Adam
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.nn import CrossEntropyLoss

In [8]:
iris=datasets.load_iris()

Podział danych

In [12]:
x = iris.data
y = iris.target
x_train,x_valid,y_train,y_valid=train_test_split(x, y, test_size=.3)

In [20]:
device = torch.device(0)
print(f'device: {device}')

device: cuda:0


In [16]:
class EcgDataset(Dataset):
    def __init__(self, x, y):
        self.x = Tensor(x).to(device)
        self.y = Tensor(y).type(torch.LongTensor).to(device)
        
    def __len__(self):
        return len(self.x)
 
    def __getitem__(self, idx):
        return [self.x[idx], self.y[idx]]

In [17]:
ds_train = EcgDataset(x_train, y_train)
ds_valid = EcgDataset(x_valid, y_valid)
dl_train = DataLoader(ds_train, batch_size=16, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size=1024, shuffle=False)

Model

In [35]:
class IrisNet(Module):
    
    def __init__(self, n_inputs, n_outputs):
        super(IrisNet, self).__init__()

        self.linear1 = Linear(n_inputs, 15)
        self.relu = ReLU()
        self.linear2 = Linear(15, n_outputs)
        self.softmax = Softmax(dim=1)
 
    def forward(self, X):
        X = self.linear1(X)
        X = self.relu(X)
        X = self.linear2(X)
        X = self.softmax(X)
        return X

In [36]:
model = IrisNet(4, 3).to(device)

Trening

In [37]:
def trainModel(model, dl_train, dl_valid, n_epoch):
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)
    
    # epoki uczenia
    for epoch in tqdm(range(n_epoch)):
        if epoch % 5 == 0:
            with torch.no_grad():
                accTrain, lossTrain = evaluateModel(model, dl_train)
                accValid, lossValid = evaluateModel(model, dl_valid)
                print(f'epoch: {epoch}, Train ACC: {accTrain}, loss: {lossTrain}, Valid ACC: {accValid}, loss: {lossValid}')

        for i, (inputs, targets) in enumerate(dl_train):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets.flatten())
            loss.backward()
            optimizer.step()

In [38]:
def evaluateModel(model, test_dl):
    with torch.no_grad():
        criterion = CrossEntropyLoss()

        x = test_dl.dataset.x
        y = test_dl.dataset.y
        
        yhat = model(x)
        losses = F.cross_entropy(yhat, y.flatten()).detach().cpu().numpy()
        
        yhat = yhat.detach().cpu().numpy()
        actuals = y.cpu().numpy()
        predictions = np.argmax(yhat, axis=1)
        
        acc = accuracy_score(actuals, predictions)
        loss = losses.mean()
        return acc, loss

In [39]:
n_epoch = 100
trainModel(model, dl_train, dl_valid, n_epoch)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, Train ACC: 0.34285714285714286, loss: 1.1117284297943115, Valid ACC: 0.3111111111111111, loss: 1.1228059530258179
epoch: 5, Train ACC: 0.34285714285714286, loss: 1.0912303924560547, Valid ACC: 0.3111111111111111, loss: 1.1022017002105713
epoch: 10, Train ACC: 0.34285714285714286, loss: 1.0770721435546875, Valid ACC: 0.3111111111111111, loss: 1.0867270231246948
epoch: 15, Train ACC: 0.34285714285714286, loss: 1.0595675706863403, Valid ACC: 0.3111111111111111, loss: 1.065768837928772
epoch: 20, Train ACC: 0.6476190476190476, loss: 1.034699559211731, Valid ACC: 0.6888888888888889, loss: 1.0336154699325562
epoch: 25, Train ACC: 0.6476190476190476, loss: 1.00166654586792, Valid ACC: 0.7111111111111111, loss: 0.9912658929824829
epoch: 30, Train ACC: 0.6476190476190476, loss: 0.9638787508010864, Valid ACC: 0.7111111111111111, loss: 0.9441419243812561
epoch: 35, Train ACC: 0.6476190476190476, loss: 0.9283139705657959, Valid ACC: 0.7111111111111111, loss: 0.9008463621139526
epoch: 40,

Zapis wag

In [40]:
w1 = model.linear1.weight.detach().cpu().numpy()
b1 = model.linear1.bias.detach().cpu().numpy()
w2 = model.linear2.weight.detach().cpu().numpy()
b2 = model.linear2.bias.detach().cpu().numpy()

In [41]:
x = ds_valid.x[0].detach().cpu().numpy()

In [43]:
np.save('ml/w1.npy', w1)
np.save('ml/b1.npy', b1)
np.save('ml/w2.npy', w2)
np.save('ml/b2.npy', b2)
np.save('ml/x.npy', x)